Let's test our combined model


In [1]:
from tensorflow.keras.models import load_model


# Specify the path to the saved model file (change this to your model's path)

type_model_path = 'CNN_Choosing_PlantDisease_64.h5'

colored_model_path = 'CNN_Colored_PlantDisease_64.h5'

grayscale_model_path = 'CNN_grayscale_PlantDisease_64.h5'

segmented_model_path = 'CNN_Segmented_PlantDisease_64.h5'

# Load the saved model

type_model = load_model(type_model_path)

colored_model = load_model(colored_model_path)

grayscale_model = load_model(grayscale_model_path)

segmented_model = load_model(segmented_model_path)

In [2]:
import os

def create_label_dictionary(folder_path):
    label_dict = {}
    
    # Get a list of all folders in the specified path
    folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    
    # Assign numerical labels to folders
    for i, folder in enumerate(folders):
        label_dict[folder] = i

    return label_dict

# Specify the path to the folder containing labeled subfolders
folder_path = 'C:\\Users\\glowi\\Desktop\\SYNC_internship\\Plant disease\\archive\\plantvillage dataset\\color'

# Create the label dictionary
label_dictionary = create_label_dictionary(folder_path)

print(label_dictionary)


{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___healthy': 5, 'Cherry_(including_sour)___Powdery_mildew': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___healthy': 9, 'Corn_(maize)___Northern_Leaf_Blight': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___healthy': 13, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15, 'Peach___Bacterial_spot': 16, 'Peach___healthy': 17, 'Pepper,_bell___Bacterial_spot': 18, 'Pepper,_bell___healthy': 19, 'Potato___Early_blight': 20, 'Potato___healthy': 21, 'Potato___Late_blight': 22, 'Raspberry___healthy': 23, 'Soybean___healthy': 24, 'Squash___Powdery_mildew': 25, 'Strawberry___healthy': 26, 'Strawberry___Leaf_scorch': 27, 'Tomato___Bacterial_spot': 28, 'Tomato___Early_blight': 29, 'Tomato___healthy': 30, 'Tomato___L

In [3]:
import cv2
import numpy as np

def classify_image(image_path):
    # Read the image using OpenCV
    img = cv2.imread(image_path)

    # Resize the image to a consistent size
    img = cv2.resize(img, (64, 64))

    # Normalize pixel values to the range [0, 1]
    img = img.astype('float32') / 64.0

    # Expand dimensions to match the model's expected input shape
    img = np.expand_dims(img, axis=0)

    # Determine the type of the image
    type_prediction = np.argmax(type_model.predict(img))

    # Classify based on the type
    if type_prediction == 0:  # Colored
        result = colored_model.predict(img)
        class_prediction = np.argmax(result)
        class_confidence = result[0][class_prediction]
        class_label = 'Colored'

    elif type_prediction == 1:  # Grayscale
        result = grayscale_model.predict(img)
        class_prediction = np.argmax(result)
        class_confidence = result[0][class_prediction]
        class_label = 'Grayscale'

    else:  # Segmented
        result = segmented_model.predict(img)
        class_prediction = np.argmax(result)
        class_confidence = result[0][class_prediction]
        class_label = 'Segmented'

    return class_label, class_prediction, class_confidence


In [4]:
# Example usage
image_path = 'archive\\plantvillage dataset\\color\\Apple___Black_rot\\0b37761a-de32-47ee-a3a4-e138b97ef542___JR_FrgE.S 2908.JPG'
label, prediction, confidence = classify_image(image_path)

# Access the class name associated with the numerical label 1
class_name_for_label = next(key for key, value in label_dictionary.items() if value == prediction)

print(f"Type: {label}, Prediction: {class_name_for_label} , Confidence: {confidence}")


Type: Colored, Prediction: Apple___Black_rot , Confidence: 0.9715620875358582
